In [ ]:
# --- Standalone Script: Comprehensive Correlation Analysis with Context ---

# Step 1: Install necessary libraries
!pip install vitaldb pandas numpy seaborn matplotlib tqdm --quiet
print("--- Libraries installed ---")

import vitaldb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
import subprocess
import warnings

# Ignore common runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- Configuration ---
SAMPLE_CASE_IDS = list(range(1, 501)) # Use the 500 cases that gave us stable results

# --- Function to process a single case (Unchanged) ---
def process_case(case_id):
    padded_id = f"{case_id:04d}"
    s3_path = f"s3://physionet-open/vitaldb/1.0.0/vital_files/{padded_id}.vital"
    local_file = f"{padded_id}.vital"
    try:
        command = ["aws", "s3", "cp", "--no-sign-request", s3_path, local_file]
        subprocess.run(command, check=True, capture_output=True, text=True)
        if not os.path.exists(local_file): return None
        vf = vitaldb.VitalFile(local_file)
        track_names = vf.get_track_names()
        numeric_tracks = [name for name in track_names if '_WAV' not in name and 'EVENT' not in name]
        if not numeric_tracks: return None
        df = vf.to_pandas(track_names=numeric_tracks, interval=1)
        if 'BIS/BIS' in df.columns:
            df = df[(df['BIS/BIS'] >= 1) & (df['BIS/BIS'] <= 100)]
        if df.empty: return None
        patient_means = df.mean(numeric_only=True).to_dict()
        patient_means['caseid'] = case_id
        return patient_means
    except Exception:
        return None
    finally:
        if os.path.exists(local_file): os.remove(local_file)

# --- Main Execution ---
print(f"Starting analysis on {len(SAMPLE_CASE_IDS)} sample cases...")
all_patient_data = []
for case_id in tqdm(SAMPLE_CASE_IDS, desc="Processing Patient Files"):
    patient_summary = process_case(case_id)
    if patient_summary:
        all_patient_data.append(patient_summary)

print(f"\nSuccessfully processed {len(all_patient_data)} cases.")

if all_patient_data:
    full_aggregated_df = pd.DataFrame(all_patient_data)
    full_aggregated_df.set_index('caseid', inplace=True)

    # --- CONTEXTUAL ANALYSIS ---
    print("\nCalculating comprehensive correlation matrix...")
    comprehensive_corr_matrix = full_aggregated_df.corr()

    if 'BIS/BIS' in comprehensive_corr_matrix:
        # 1. TEST FOR SPARSITY: Calculate the pairwise observation count
        print("Calculating pairwise observation counts...")
        observation_counts = full_aggregated_df.notna().T.dot(full_aggregated_df.notna())

        # Combine correlation and counts into a single DataFrame for analysis
        corr_with_bis = comprehensive_corr_matrix['BIS/BIS'].dropna()
        counts_with_bis = observation_counts['BIS/BIS'].reindex(corr_with_bis.index)

        contextual_corr_df = pd.DataFrame({
            'Correlation': corr_with_bis,
            'PatientCount': counts_with_bis
        }).sort_values(by='Correlation', ascending=False)

        print("\n--- Top 20 Most Correlated Features with BIS/BIS (with Patient Counts) ---")
        print(contextual_corr_df.head(20))

        print("\n--- Top 20 Most Negatively Correlated Features with BIS/BIS (with Patient Counts) ---")
        print(contextual_corr_df.tail(20))

        # 2. TEST FOR COLLINEARITY: Visualize the most extreme correlations
        print("\n--- Visualizing Top Correlations to Check for Collinearity ---")

        # Get the top 3 positive and top 3 negative correlations (excluding the perfect +/- 1.0)
        reliable_corrs = contextual_corr_df[(contextual_corr_df['Correlation'] < 0.999) & (contextual_corr_df['Correlation'] > -0.999)]
        top_positive_features = reliable_corrs.head(3).index.tolist()
        top_negative_features = reliable_corrs.tail(3).index.tolist()
        features_to_plot = top_positive_features + top_negative_features

        for feature in features_to_plot:
            plt.figure(figsize=(8, 6))
            sns.scatterplot(data=full_aggregated_df, x=feature, y='BIS/BIS', alpha=0.5)
            plt.title(f'BIS/BIS vs {feature}\n(Correlation: {contextual_corr_df.loc[feature, "Correlation"]:.2f}, Patients: {int(contextual_corr_df.loc[feature, "PatientCount"])})')
            plt.grid(True)
            plt.show()

    else:
        print("\n'BIS/BIS' track not found in the processed data.")
else:
    print("No data was successfully processed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 32.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
--- Libraries installed ---
Starting analysis on 500 sample cases...


Processing Patient Files:   0%|          | 0/500 [00:00<?, ?it/s]


Successfully processed 0 cases.
No data was successfully processed.
